In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import poisson
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import plotly.express as px


In [2]:
def sqrt_func(i, j):
    time.sleep(1)
    return i + j


Parallel(n_jobs=2)(delayed(sqrt_func)(i, 5)
                   for i in range(5))


[5, 6, 7, 8, 9]

## Functions

In [3]:
I = 48  # Number of intervals
d = 5  # length of interval
x = np.random.choice(5, I)
N = sum(x)
precision = 0.9999
beta = 9  # Average service time for a patient
no_show = 0 # Fraction of scheduled patients not showing up
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4

print(N)

87


In [4]:
def calcExponentialLimit(mu):
    return int(max(mu+4*mu**0.5, 100))

In [5]:
def binomCoeff(k, i):
  return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))


def binomPMF(k, i, m, add_v, no_show):
  return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)

In [6]:
# Distribution to calculate service time of patients
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.
def calculate_p(beta, size, precision=0.9999):  # Poisson distribution
    k = 0
    p = []

    while sum(p) < precision:  # fill accurate values up to precision limit
        p.append(poisson.pmf(k, beta))
        k += 1

    while len(p) < size:  # fill the rest of the values with 0
        p.append(0)
    return p, k

##########
## TEST ##
##########

probS = calcExponentialLimit(beta*N)+1
print(probS)
calculate_p(beta, probS, precision)


895


([0.00012340980408667956,
  0.0011106882367801166,
  0.004998097065510523,
  0.014994291196531574,
  0.033737155192196056,
  0.06072687934595293,
  0.09109031901892926,
  0.1171161244529091,
  0.13175564000952278,
  0.13175564000952278,
  0.11858007600857066,
  0.09702006218883041,
  0.0727650466416229,
  0.050375801521123684,
  0.03238444383500792,
  0.01943066630100472,
  0.010929749794315179,
  0.005786338126402172,
  0.0028931690632010803,
  0.0013704485036215655,
  0.0006167018266297063,
  0.0002643007828413016,
  0.00010812304752598687,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [7]:
def calculate_v(p, beta, precision, precision_limit, N, d, no_show=0):
  count = precision_limit
  limit = calcExponentialLimit(beta*N) + 1
  v = np.zeros((N+1, limit+d))
  add_v = np.zeros((N+1, limit+d))

  add_v[0][0] = 1
  for k in range(1, N+1):
    limit = calcExponentialLimit(beta*k)
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      z = 0
      while z <= count:
        add_v[k][i] += p[z] * add_v[k-1][i-z]
        z += 1
      sum_v += add_v[k][i]
      i += 1

  for k in range(N+1):
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      for m in range(k+1):
        v[k][i] += binomPMF(k, i, m, add_v, no_show)
      sum_v += v[k][i]
      i += 1

  return v, limit

##########
## TEST ##
##########

p, precision_limit = calculate_p(beta, probS, precision)
calculate_v(p, beta, precision, precision_limit, N, d, no_show)


(array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.23409804e-04, 1.11068824e-03, 4.99809707e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.52299797e-08, 2.74139635e-07, 2.46725672e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.02573831e-07, 5.21365308e-08, 2.51443352e-08, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 894)

In [8]:
def calculateProbabilities(x, precision, limit, v, I, d):
  p_plus = np.zeros((I+1, limit+d+1))
  p_min = np.zeros((I+1, limit+d+1))

  # Constraint 1
  p_min[0][0] = 1

  # Constraint 2
  sum_p = 0
  i = 0
  while sum_p < precision and i <= limit:
    p_plus[0][i] = v[x[0]][i]
    sum_p += p_plus[0][i]
    i += 1

  for t in range(1, I+1):  # calculate p_min and p_plus iteratively
    # Constraint 3
    for k in range(d+1):
      # probability of amount of work = 0 just before the start of t equals the cummulative probablity of amount of work less or equal to duration of the interval just at the start of the previous interval, t-1
      p_min[t][0] += p_plus[t-1][k]
    # Constraint 4
    for i in range(1, limit+1):

      # probability of amount of work = i just before the start of interval t equals the probablity of amount of work exceeding the duration of the interval by i just at the start of the previous interval, t-1
      p_min[t][i] = p_plus[t-1][i+d]

    # Constraint 5
    if t != I:  # I or I+1
      for i in range(limit+1):
        for j in range(i+1):
          p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

  return p_min, limit

##########
## TEST ##
##########

v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)
p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)


In [9]:
def calcTardiness(p_min, limit, I):
  tardiness = 0
  # print(p_min[I])
  for k in range(limit):
    tardiness += k * p_min[I][k]  # I+1
  return tardiness


def calcIdletime(I, d, tardiness, N, no_show, beta):
  return (I * d) + tardiness - (N * (1 - no_show) * beta)  # I-1?


def calcWaitingtime(p_min, x, p, limit, I, N):
  w = np.zeros((I+1, N+1, limit+1))
  waitingtime = 0

  for t in range(0, I):
    if x[t] > 0:
      for k in range(limit):
        w[t][0][k] = p_min[t][k]
    if x[t] > 1:
      for i in range(1, x[t]+1):
        for k in range(limit+1):
          for j in range(k+1):
            w[t][i][k] += w[t][i-1][j] * p[k-j]

  for t in range(0, I):
    for i in range(0, x[t]):
      for k in range(limit+1):
        waitingtime += w[t][i][k] * k

  waitingtime /= N
  return waitingtime

##########
## TEST ##
##########

calcWaitingtime(p_min, x, p, limit, I, N)


264.6152716142531

In [10]:
def calcFracExcess(p_min, I):
  fracExcess = 0
  t = I+1
  for j in range(1, len(p_min[t])):
    fracExcess += p_min[t][j]
  fracExcess *= 100
  return fracExcess


##########
## TEST ##
##########

# calcFracExcess(p_min, I)


<div>
<img src="https://github.com/witusj/obp/blob/master/images/fitness2.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/fitness.png?raw=true" width="450">
</div>

In [11]:
def calcFitness(x, beta, precision, limit, v, N, no_show, I, d, eind, alpha_W, alpha_I, alpha_T):
  tic = time.perf_counter()
  p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)
  toc = time.perf_counter()
  probT = toc-tic

  # Tardiness calcs
  tic = time.perf_counter()
  tardiness = calcTardiness(p_min, limit, I)
  toc = time.perf_counter()
  tardT = toc-tic

  # Idle time calcs new array of given shape and type, filled with zeros.
  tic = time.perf_counter()
  idletime = calcIdletime(I, d, tardiness, N, no_show, beta)
  toc = time.perf_counter()
  idletimeT = toc-tic

  # Waiting time calcs
  tic = time.perf_counter()
  waitingtime = calcWaitingtime(p_min, x, p, limit, I, N)
  toc = time.perf_counter()
  waitingtimeT = toc-tic

  objVal = alpha_W*waitingtime + alpha_I*idletime + alpha_T*tardiness

  print(f"Schedule: {x},\nN: {sum(x)},\nObjective value: {objVal},\nProb calculation time: {probT:.6f} sec,\nWaiting time (timer): {waitingtime} ({waitingtimeT:.6f} sec),\nIdle time (timer): {idletime} ({idletimeT:.6f} sec),\nTardiness (timer): {tardiness} ({tardT:.6f} sec)\n")

  # Collect into a dictionary
  results = {'p_min': p_min, 'waitingTime': waitingtime,
            'idleTime': idletime, 'tardiness': tardiness, 'objVal': objVal}

  if eind == 1:
    fracExcess = calcFracExcess(p_min, I)
    results['fracExcess'] = fracExcess

  return results['objVal']

##########
## TEST ##
##########


p, precision_limit = calculate_p(beta, probS, precision)
v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)



results = calcFitness(x, beta, precision, limit, v, N, no_show,
                  I, d, eind, alpha_W, alpha_I, alpha_T)
results


Schedule: [3 1 1 4 2 4 1 0 0 2 0 3 0 2 2 3 1 3 4 0 0 1 3 1 2 0 1 1 1 2 4 4 1 4 1 1 2
 0 2 0 4 3 4 4 0 2 3 0],
N: 87,
Objective value: 321.1068229141437,
Prob calculation time: 15.657939 sec,
Waiting time (timer): 264.6152716142531 (28.092457 sec),
Idle time (timer): -3.2321428859291927 (0.000004 sec),
Tardiness (timer): 539.7678571140708 (0.000390 sec)



321.1068229141437

In [12]:
def createDist(arr, reverse=False):
    minX, maxX = arr.min(), arr.max()
    minY, maxY = 0, 1
    if(reverse):
        minY, maxY = maxY, minY
    # Scale to values between 0, 1 and make total sum equal to 1
    arrScaled = np.interp(arr, (minX, maxX), (minY, maxY))
    arrNorm = [x / arrScaled.sum() for x in arrScaled]
    return(arrNorm)
    

##########
## TEST ##
##########

arr = np.random.choice(10, 5)
dist1 = createDist(arr)
dist2 = createDist(arr, reverse=True)
arr, dist1, sum(dist1), dist2, sum(dist2)


(array([3, 8, 1, 7, 0]),
 [0.15789473684210525,
  0.42105263157894735,
  0.05263157894736842,
  0.3684210526315789,
  0.0],
 1.0,
 [0.23809523809523808,
  0.0,
  0.3333333333333333,
  0.047619047619047616,
  0.38095238095238093],
 1.0)

<div>
<img src="https://github.com/witusj/obp/blob/master/images/crossover.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/crossover2.png?raw=true" width="450">
</div>

In [13]:
# NEW
def crossover(parent1, parent2, nSwaps):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    I = len(child1)
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    p1 = createDist(child1)
    p2 = createDist(child2, reverse=True)
    
    # Swap patients parent1 to parent 2
    n1 = nSwaps
    while(n1 > 0):
        i = np.random.choice(I, 1, p=p1)
        j = np.random.choice(I, 1, p=p2)
        child1[i] += -1
        child2[j] += 1
        if(child1[i] == 0):
            p1 = createDist(child1) # If the number of patients in interval i reaches zero, this interval should not be selected again.
        n1 -= 1
    
    # Swap patients parent2 to parent 1
    p2 = createDist(child2)
    p1 = createDist(child1, reverse=True)
    
    n2 = nSwaps
    while(n2 > 0):
        i = np.random.choice(I, 1, p=p2)
        j = np.random.choice(I, 1, p=p1)
        child2[i] += -1
        child1[j] += 1
        if(child2[i] == 0):
            p2 = createDist(child2)
        n2 -= 1
        
    return(child1, child2)

##########
## TEST ##
##########

testI = 10
parent1 = np.random.choice(5, testI)
parent2 = np.random.choice(5, testI)

print(parent1, parent2)
nSwaps = 3
children = crossover(parent1, parent2, nSwaps)
children


[4 2 4 1 2 3 4 0 1 3] [0 4 4 4 3 0 0 1 2 1]


(array([3, 1, 3, 1, 2, 3, 4, 1, 3, 3]), array([1, 4, 2, 3, 3, 1, 1, 1, 2, 1]))


<div>
    <img src="https://github.com/witusj/obp/blob/master/images/mutation.png?raw=true" width="450">
    <img src="https://github.com/witusj/obp/blob/master/images/mutation1.png?raw=true" width="450">
</div>

In [14]:
def mutate(child, rate):
    if np.random.rand() < rate:
        x = len(child)
        i, j = np.random.choice(x, 2, replace=False).astype(int)
        a, b = child[i], child[j]
        child[i], child[j] = b, a
    return(child)

##########
## TEST ##
##########

child = children[0].copy()
print(child)
child = mutate(child, 1)
print(child)


[3 1 3 1 2 3 4 1 3 3]
[3 1 3 3 2 3 4 1 1 3]


## Classes

#### Suggesties: [get / set methods](https://www.geeksforgeeks.org/getter-and-setter-in-python/) voor parameters / met tests / nuttig voor optimaliseren hyper parameters, bv mbv bayesian stats

Create a data object to store and generate initial data.

In [15]:
class Data:
    def __init__(self, I=10, N=12, d=5, popSize=5, beta=9, precision=0.9999, no_show=0, eind=0, alpha_I=0.2, alpha_T=0.3, alpha_W=0.5, nSwaps=4, mutationRate=0.1):
        self.I = I
        self.N = N
        self.d = d
        self.popSize = popSize
        self.beta = beta
        self.precision = precision
        self.no_show = no_show
        self.probSize = calcExponentialLimit(self.beta*self.N)+1
        self.p, self.precision_limit = self.__calculate_p()
        self.v, self.limit = self.__calculate_v()
        self.eind = eind
        self.alpha_I, self.alpha_T, self.alpha_W = alpha_I, alpha_T, alpha_W
        self.nSwaps, self.mutationRate = nSwaps, mutationRate
    
    def get_initial_data(self): return np.array((self.I, self.N, self.d, self.beta, self.popSize, self.eind, self.alpha_I, self.alpha_T, self.alpha_W, self.nSwaps, self.mutationRate),
                                                dtype={'names': ('I', 'N', 'd', 'beta', 'popSize', 'eind', 'alpha_I', 'alpha_T', 'alpha_W', 'nSwaps', 'mutationRate'),
                                                'formats': ('i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'f8', 'f8', 'f8', 'i4', 'f8')})

    def get_prob_data(self): return np.array((self.precision, self.no_show, self.probSize, self.precision_limit, self.limit),
                                                dtype={'names': ('precision', 'no_show', 'probSize', 'precision_limit', 'limit'),
                                                'formats': ('f8', 'f8', 'i4', 'f8', 'i4')})

    def calculate_p(self):  return calculate_p(self.beta, self.probSize, self.precision)
    __calculate_p = calculate_p
    
    def calculate_v(self): return calculate_v(self.p, self.beta, self.precision, self.precision_limit, self.N, self.d, self.no_show)
    __calculate_v = calculate_v
    

##########
## TEST ##
##########

newDataObject = Data(I, N)
initData = newDataObject.get_initial_data()
initData[['N', 'popSize']]


array((87, 5),
      dtype={'names': ['N', 'popSize'], 'formats': ['<i4', '<i4'], 'offsets': [4, 16], 'itemsize': 60})

<img src="https://github.com/witusj/obp/blob/master/images/population.png?raw=true" width="600">

Create a random individual. An individual is in this case one schedule.

In [16]:
class Schedule:
    def __init__(self, dataObject):
        self.I = dataObject.get_initial_data()['I']  # Number of intervals
        self.N = dataObject.get_initial_data()['N']  # Number of patients
        self.x = np.zeros(self.I).astype(int)# Empty schedule

    def create_random_schedule(self):
        for i in range(self.N):
            i = np.random.choice(self.I, 1)  # Interval ID
            self.x[i] = self.x[i] + 1

    def create_bw_schedule(self):
        self.x = np.ones(self.I).astype(int)
        self.x[0] = 2
        checkSum = np.sum(self.x)
        print(f'checkSum = {checkSum}')
        diff = checkSum - self.N
        print(f'diff = {diff}')
        if diff > 0:
            i = np.random.choice(range(1, self.I), diff, replace=False)
            print(f'i = {i}')
            self.x[i] = self.x[i] - 1
        if diff < 0:
            i = np.random.choice(range(1, self.I), -diff, replace=False)
            print(f'i = {i}')
            self.x[i] = self.x[i] + 1
    def get_schedule(self): return(self.x)
    def print_schedule(self): print(self.x)


##########
## TEST ##
##########

dataObject = Data(I=I, N=N, d=d)
scheduleObject = Schedule(dataObject)
scheduleObject.print_schedule()
scheduleObject.create_random_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())
scheduleObject.create_bw_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
[3 1 2 5 2 0 0 1 5 1 5 3 1 0 2 3 2 1 3 2 1 3 0 2 0 2 3 1 1 2 1 0 1 2 0 3 3
 1 3 2 4 0 2 0 5 1 1 1]
87
checkSum = 49
diff = -38
i = [43 35 26 33 11 44 13 15 36 27  5 34 20 12 21 45 47  3 22 10 16 28 18 38
 14 29  2 32  4  7  6 46 42 31  1 23 40 37]
[2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 1 2 1 2 2 2 2 1 1 2 2 2 2 1 2 2 2 2 2 2
 2 2 1 2 1 2 2 2 2 2 2]
87


Create population of individuals

In [17]:
#NEW
class Population:
    global bestRank
    bestRank = []
    def __init__(self, dataObject, individuals=None):
        self.dataObject = dataObject
        self.params = self.dataObject.get_initial_data()
        self.prob_params = self.dataObject.get_prob_data()
        self.p, self.v = self.dataObject.p, self.dataObject.v
        self.individuals = individuals
        if self.individuals == None: self.__create_new_individuals() 
        self.fitnessScores = []

    def create_new_individuals(self):
        self.individuals = []
        for i in range(self.params['popSize']):
            x = Schedule(self.dataObject)
            x.create_random_schedule()
            x = x.get_schedule()
            self.individuals.append(x)
            
    __create_new_individuals = create_new_individuals
    
    def get_individuals(self): return(self.individuals)
    
    def calc_fitness_scores(self):
            beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[['beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
            precision, limit, no_show = self.prob_params[['precision', 'limit', 'no_show']].tolist()
            self.fitnessScores = np.array(Parallel(n_jobs=6)(delayed(calcFitness)(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T) for x in self.individuals))
            
            
    def apply_crossover(self):
        newIndividuals = []
        nSwaps = self.params['nSwaps']*1 ## Times 1 because otherwise refers to memorypoint
        mutationRate = self.params['mutationRate']
        self.population_dist = createDist(self.fitnessScores, reverse=True)
        
        for i in range(self.params['popSize']):
            print(f'Crossover {i}')
            r1, r2 = np.random.choice(self.params['popSize'], 2, replace=False, p=self.population_dist)
            parent1, parent2 = self.individuals[r1], self.individuals[r2]
            child1, child2 = crossover(parent1, parent2, nSwaps)
            newIndividuals.append(mutate(child1, mutationRate))
            newIndividuals.append(mutate(child2, mutationRate))
            
        beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[[
            'beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
        precision, limit, no_show = self.prob_params[[
            'precision', 'limit', 'no_show']].tolist()
        self.fitnessScores = np.array(Parallel(n_jobs=6)(delayed(calcFitness)(x, beta=beta, precision=precision, limit=limit, v=self.v,
                                                                              N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T) for x in newIndividuals))
        tempList = list(range(self.params['popSize']*2))
        zipped = list(zip(*sorted(zip(self.fitnessScores, tempList))))
        self.fitnessScores = np.array(list(zipped[0][:self.params['popSize']]))
        tempList = zipped[1][:self.params['popSize']]
        self.individuals = list(map(newIndividuals.__getitem__, tempList))
        bestRank.append([self.fitnessScores[0], self.individuals[0]])
        print(self.fitnessScores, '\n', self.individuals)


## Tests

In [18]:
##########
## TEST ##
##########

bestRank = []
dataObject = Data()
populationObject1 = Population(dataObject)
populationObject1.fitnessScores
populationObject1.calc_fitness_scores()
print(populationObject1.fitnessScores)


Schedule: [1 5 1 1 1 2 0 1 0 0],
N: 12,
Objective value: 35.73098537669306,
Prob calculation time: 0.110831 sec,
Waiting time (timer): 36.63641641315435 (0.088874 sec),
Idle time (timer): 0.025554340231778383 (0.000002 sec),
Tardiness (timer): 58.02555434023178 (0.000061 sec)

Schedule: [2 0 1 1 0 2 2 1 2 1],
N: 12,
Objective value: 29.883851689477368,
Prob calculation time: 0.110253 sec,
Waiting time (timer): 24.952192621505006 (0.101381 sec),
Idle time (timer): 0.015510757449732182 (0.000002 sec),
Tardiness (timer): 58.01551075744974 (0.000060 sec)

Schedule: [1 0 1 2 1 1 2 2 0 2],
N: 12,
Objective value: 31.15915895413766,
Prob calculation time: 0.129446 sec,
Waiting time (timer): 25.74202617768961 (0.102964 sec),
Idle time (timer): 1.7762917305857115 (0.000002 sec),
Tardiness (timer): 59.77629173058571 (0.000062 sec)

Schedule: [2 1 1 0 0 3 1 2 2 0],
N: 12,
Objective value: 30.90737305903456,
Prob calculation time: 0.115667 sec,
Waiting time (timer): 27.017643448173143 (0.113296 se

In [19]:
individuals2 = populationObject1.individuals.copy()
populationObject2 = Population(dataObject, individuals=individuals2)
print(populationObject1.individuals, '\n', populationObject2.individuals)

[array([1, 0, 1, 2, 1, 1, 2, 2, 0, 2]), array([1, 5, 1, 1, 1, 2, 0, 1, 0, 0]), array([2, 2, 0, 0, 0, 1, 2, 2, 2, 1]), array([2, 0, 1, 1, 0, 2, 2, 1, 2, 1]), array([2, 1, 1, 0, 0, 3, 1, 2, 2, 0])] 
 [array([1, 0, 1, 2, 1, 1, 2, 2, 0, 2]), array([1, 5, 1, 1, 1, 2, 0, 1, 0, 0]), array([2, 2, 0, 0, 0, 1, 2, 2, 2, 1]), array([2, 0, 1, 1, 0, 2, 2, 1, 2, 1]), array([2, 1, 1, 0, 0, 3, 1, 2, 2, 0])]


In [20]:
populationObject2.calc_fitness_scores()
populationObject2.apply_crossover()
populationObject2.individuals


Schedule: [1 0 1 2 1 1 2 2 0 2],
N: 12,
Objective value: 31.15915895413766,
Prob calculation time: 0.127894 sec,
Waiting time (timer): 25.74202617768961 (0.105781 sec),
Idle time (timer): 1.7762917305857115 (0.000002 sec),
Tardiness (timer): 59.77629173058571 (0.000066 sec)

Schedule: [1 5 1 1 1 2 0 1 0 0],
N: 12,
Objective value: 35.73098537669306,
Prob calculation time: 0.133570 sec,
Waiting time (timer): 36.63641641315435 (0.097882 sec),
Idle time (timer): 0.025554340231778383 (0.000002 sec),
Tardiness (timer): 58.02555434023178 (0.000065 sec)

Schedule: [2 0 1 1 0 2 2 1 2 1],
N: 12,
Objective value: 29.883851689477368,
Prob calculation time: 0.137114 sec,
Waiting time (timer): 24.952192621505006 (0.100675 sec),
Idle time (timer): 0.015510757449732182 (0.000004 sec),
Tardiness (timer): 58.01551075744974 (0.000127 sec)

Schedule: [2 1 1 0 0 3 1 2 2 0],
N: 12,
Objective value: 30.90737305903456,
Prob calculation time: 0.122779 sec,
Waiting time (timer): 27.017643448173143 (0.118467 se

[array([2, 2, 1, 0, 0, 1, 0, 2, 1, 3]),
 array([1, 1, 2, 0, 0, 3, 1, 1, 2, 1]),
 array([2, 1, 1, 0, 0, 3, 0, 3, 1, 1]),
 array([1, 0, 1, 1, 2, 2, 2, 0, 1, 2]),
 array([2, 1, 2, 1, 1, 0, 2, 2, 0, 1])]

In [21]:
# Initial population
bestRank = []
dataObject = Data()
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
individuals = populationZero.individuals.copy()
generations = 3

# Genetic algorithm loop
for i in range(generations):
    print(f'Generation: {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.calc_fitness_scores()
    populationObject.apply_crossover()
    individuals = populationObject.individuals


Schedule: [0 1 1 2 2 1 0 3 1 1],
N: 12,
Objective value: 34.70115492493048,
Prob calculation time: 0.099400 sec,
Waiting time (timer): 29.567367719561876 (0.082268 sec),
Idle time (timer): 5.034942130299072 (0.000010 sec),
Tardiness (timer): 63.03494213029908 (0.000057 sec)

Schedule: [0 0 3 0 3 0 1 3 1 1],
N: 12,
Objective value: 39.1892088031184,
Prob calculation time: 0.101512 sec,
Waiting time (timer): 33.644514957977194 (0.106675 sec),
Idle time (timer): 9.933902648259604 (0.000002 sec),
Tardiness (timer): 67.9339026482596 (0.000057 sec)

Schedule: [1 1 2 1 3 0 0 0 0 4],
N: 12,
Objective value: 30.53778209280383,
Prob calculation time: 0.097623 sec,
Waiting time (timer): 26.236203757063777 (0.106869 sec),
Idle time (timer): 0.03936042854388688 (0.000002 sec),
Tardiness (timer): 58.03936042854389 (0.000057 sec)

Schedule: [0 0 1 3 1 0 4 1 0 2],
N: 12,
Objective value: 38.85278996085265,
Prob calculation time: 0.097405 sec,
Waiting time (timer): 32.888035112636736 (0.106945 sec),
Id

In [22]:

bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF


,Score,Schedule
0,29.083543,"[1, 1, 1, 1, 2, 0, 1, 0, 2, 3]"
1,29.707585,"[1, 1, 1, 1, 1, 1, 2, 1, 1, 2]"
2,28.791402,"[1, 2, 1, 0, 0, 0, 2, 1, 2, 3]"


In [23]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [1 0 1 1 0 0 0 0 1 0 0 1 1 1 0 0 1 1 0 0 1 0 2 1 0 0 0 1 0 0 1 0 0 0 0 0 1
 1 2 1 0 0 0 0 1 1 1 1],
N: 24,
Objective value: 20.736777452422295,
Prob calculation time: 1.676145 sec,
Waiting time (timer): 7.00505852792424 (0.168756 sec),
Idle time (timer): 45.891256735420995 (0.000002 sec),
Tardiness (timer): 21.891256735421003 (0.000099 sec)

Schedule: [0 2 0 1 0 0 0 0 0 2 0 0 0 1 1 0 0 0 1 1 0 0 0 1 1 0 1 3 0 0 2 1 0 1 0 0 0
 1 0 1 0 0 0 0 0 1 1 1],
N: 24,
Objective value: 18.59036538819286,
Prob calculation time: 1.675537 sec,
Waiting time (timer): 11.268943658120854 (0.368513 sec),
Idle time (timer): 39.47131320824087 (0.000007 sec),
Tardiness (timer): 15.47131320824086 (0.000099 sec)

Schedule: [0 0 1 0 2 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 2 1 0 1 1 1 0 0 0 1 0 0 1 0 1
 0 3 1 0 1 1 0 0 2 1 0],
N: 24,
Objective value: 32.145500558771545,
Prob calculation time: 1.677002 sec,
Waiting time (timer): 14.032636524601239 (0.369484 sec),
Idle time (timer): 60.22074324821841 (0.00000

In [24]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score', hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='LimeGreen')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}p.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [25]:
scheduleObject = Schedule(dataObject)
scheduleObject.create_bw_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())
x = scheduleObject.get_schedule()
calcFitness(x, beta, precision, limit, v, N, no_show,
            I, d, eind, alpha_W, alpha_I, alpha_T)


checkSum = 49
diff = 25
i = [41 23 38 31 10 35  6 15 19 32 33 24 22  1 25 28 39  7  4 20  8  3 40 46
  5]
[2 0 1 0 0 0 0 0 0 1 0 1 1 1 1 0 1 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 1 0 1
 1 0 0 0 0 1 1 1 1 0 1]
24
Schedule: [2 0 1 0 0 0 0 0 0 1 0 1 1 1 1 0 1 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 1 0 1
 1 0 0 0 0 1 1 1 1 0 1],
N: 24,
Objective value: 17.031433883333342,
Prob calculation time: 15.510803 sec,
Waiting time (timer): 7.449242105156048 (0.798707 sec),
Idle time (timer): 39.41956173545154 (0.000003 sec),
Tardiness (timer): 15.419561735451532 (0.000465 sec)



17.031433883333342

In [26]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 10

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [0 0 1 0 0 1 0 0 0 1 1 1 2 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 2
 1 1 2 0 1 1 2 1 0 0 0],
N: 24,
Objective value: 37.02713902915565,
Prob calculation time: 1.657113 sec,
Waiting time (timer): 17.367227253405844 (0.325817 sec),
Idle time (timer): 66.13374687965552 (0.000002 sec),
Tardiness (timer): 42.13374687965551 (0.000099 sec)

Schedule: [1 2 0 0 0 1 0 2 0 0 1 0 0 1 0 0 1 0 2 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0
 1 1 0 2 0 2 0 1 1 1 1],
N: 24,
Objective value: 29.71260164779082,
Prob calculation time: 1.658966 sec,
Waiting time (timer): 9.003385723646863 (0.405823 sec),
Idle time (timer): 59.51874559722012 (0.000007 sec),
Tardiness (timer): 35.51874559722012 (0.000102 sec)

Schedule: [1 1 0 0 0 3 0 0 0 1 1 0 1 0 2 2 0 1 0 0 1 0 0 0 1 1 0 1 0 1 2 0 0 0 1 0 0
 0 0 0 1 0 0 0 0 0 1 1],
N: 24,
Objective value: 15.383098892515294,
Prob calculation time: 1.708064 sec,
Waiting time (timer): 14.055907071079686 (0.365929 sec),
Idle time (timer): 32.267893440139034 (0.000002

In [27]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='RoyalBlue')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}p.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [28]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 20

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 1 1 2 1 0 0 0 1 1 0 0 0 0 1 0 0
 1 1 1 1 0 2 0 2 0 0 1],
N: 24,
Objective value: 25.08319822515241,
Prob calculation time: 1.642967 sec,
Waiting time (timer): 9.943497287486604 (0.238242 sec),
Idle time (timer): 51.17633218359629 (0.000002 sec),
Tardiness (timer): 27.17633218359628 (0.000136 sec)

Schedule: [0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 2 1 0 1 0 0 1 1 1 1 2 0 0 1 0 0 0 0 0 0 1
 0 2 1 1 2 0 1 1 1 0 0],
N: 24,
Objective value: 32.00917900161977,
Prob calculation time: 1.589082 sec,
Waiting time (timer): 16.17218866180676 (0.309817 sec),
Idle time (timer): 58.56717256149511 (0.000002 sec),
Tardiness (timer): 34.5671725614951 (0.000107 sec)

Schedule: [0 0 1 0 2 0 1 1 0 1 1 1 1 0 0 1 0 1 2 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 1 0 0
 0 1 0 0 1 2 0 0 0 0 2],
N: 24,
Objective value: 18.04709821640848,
Prob calculation time: 1.591631 sec,
Waiting time (timer): 12.018240429316391 (0.310957 sec),
Idle time (timer): 38.066336741136524 (0.000002 se

In [29]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='Tomato')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}p.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [30]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 2 0 1 0 1 0 1 1 0 0 1 1 1 2 0 0 0 1 0 0 1
 0 0 3 0 0 1 0 1 0 1 1],
N: 24,
Objective value: 26.061649079057418,
Prob calculation time: 1.638627 sec,
Waiting time (timer): 13.712557441252258 (0.299802 sec),
Idle time (timer): 50.2943768375942 (0.000004 sec),
Tardiness (timer): 26.294376837594182 (0.000235 sec)

Schedule: [1 1 1 0 2 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 1 2 1 1 0 0 0 0
 1 0 0 1 0 1 2 0 1 0 1],
N: 24,
Objective value: 19.943633467144025,
Prob calculation time: 1.702774 sec,
Waiting time (timer): 8.526409433181461 (0.257919 sec),
Idle time (timer): 43.555116156452414 (0.000002 sec),
Tardiness (timer): 19.55511615645239 (0.000100 sec)

Schedule: [1 0 0 0 1 1 1 0 0 1 1 1 0 0 1 0 3 0 0 0 0 0 1 0 1 0 0 2 1 1 0 2 0 0 0 0 0
 0 0 2 1 1 0 1 0 0 0 0],
N: 24,
Objective value: 11.234280128098325,
Prob calculation time: 1.643640 sec,
Waiting time (timer): 10.239400119013581 (0.396596 sec),
Idle time (timer): 27.89753346748816 (0.0000

In [31]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='DarkOrange')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}p.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [32]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [1 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 2 1 1 1 1 0 1 1 0 0 1 2 1 0 1 1
 0 0 1 0 0 1 0 0 1 1 0],
N: 24,
Objective value: 28.74653904911928,
Prob calculation time: 1.613968 sec,
Waiting time (timer): 19.237626042497954 (0.155267 sec),
Idle time (timer): 51.08581438686684 (0.000002 sec),
Tardiness (timer): 27.08581438686683 (0.000109 sec)

Schedule: [1 1 0 0 1 0 1 0 0 0 1 1 1 0 2 0 1 1 1 0 1 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 1
 2 1 0 2 0 0 0 1 0 0 0],
N: 24,
Objective value: 12.57194959314277,
Prob calculation time: 1.609601 sec,
Waiting time (timer): 11.329106098186648 (0.229449 sec),
Idle time (timer): 29.400511923113527 (0.000002 sec),
Tardiness (timer): 5.400511923113514 (0.000110 sec)

Schedule: [0 2 0 0 0 1 0 1 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 1 1 0 0 2 0 0 0 0 0 1 0 1 1
 0 0 0 0 0 0 0 1 1 1 3],
N: 24,
Objective value: 28.267530795784367,
Prob calculation time: 1.617584 sec,
Waiting time (timer): 7.601416743139787 (0.268893 sec),
Idle time (timer): 58.0449401642141 (0.000002 

In [33]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='DarkCyan')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}p.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")
